In [ ]:
import pytest
from unittest.mock import patch, MagicMock
from rss_collector.feed_parser import retry_with_backoff

def test_retry_with_backoff_success():
    """
    Test that retry_with_backoff succeeds after a few failed attempts.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to fail twice before succeeding
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        # First two attempts fail, third succeeds
        mock_fetch.side_effect = [None, None, "<rss>Valid Feed</rss>"]

        # Mock feedparser.parse to return a valid feed
        with patch("rss_collector.feed_parser.feedparser.parse") as mock_parse:
            mock_parse.return_value = MagicMock(feed={"title": "Test Feed"}, entries=[])

            # Call the function
            parsed_feed = retry_with_backoff(test_url, retries=3)

            # Debugging outputs
            print(f"Mock Fetch Call Count: {mock_fetch.call_count}")
            print(f"Parsed Feed: {parsed_feed}")

            # Assertions
            assert mock_fetch.call_count == 3, f"Expected 3 retries, got: {mock_fetch.call_count}"
            assert parsed_feed is not None, "Expected parsed_feed to be not None."
            assert parsed_feed.feed["title"] == "Test Feed"



def test_retry_with_backoff_failure():
    """
    Test that retry_with_backoff returns None after all retries fail.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to always fail
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        mock_fetch.return_value = None

        # Call the function
        parsed_feed = retry_with_backoff(test_url, retries=3)

        # Assertions
        assert mock_fetch.call_count == 3, "Expected 3 retries, got: {mock_fetch.call_count}"
        assert parsed_feed is None, "Expected parsed_feed to be None."



In [ ]:
def retry_with_backoff(feed_url, retries=3, delay=5):
    """
    Retries fetching an RSS feed with exponential backoff.

    Args:
        feed_url (str): The URL of the RSS feed.
        retries (int): The maximum number of retry attempts.
        delay (int): The initial delay in seconds before retrying.

    Returns:
        dict: Parsed feed content if successful, None otherwise.
    """
    for attempt in range(1, retries + 1):
        print(f"Attempt {attempt} to fetch feed: {feed_url}")
        response = fetch_feed_content_with_proxy(feed_url)
        print(f"Response on attempt {attempt}: {response}")
        if response:
            feed = feedparser.parse(response)
            print(f"Parsed Feed: {feed}")
            if feed and not feed.bozo:  # Ensure the feed is valid
                return feed
        print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        delay *= 2  # Exponential backoff
    print(f"Failed to fetch feed after {retries} attempts: {feed_url}")
    return None
